In [ ]:
import os
import glob
from skimage import io
import matplotlib.pyplot as plt

from skimage.measure import regionprops
from skimage.morphology import label
import numpy as np

from pathlib import Path
import imageio

In [ ]:
labels_path = r'C:\Users\Pau\Desktop\Neurosphere Annotations\export\labels'
images_path = r'C:\Users\Pau\Desktop\Neurosphere Annotations\export\images'

all_images =[]
for file in glob.glob(images_path + "\*.tif"):
    all_images.append(os.path.basename(file))

In [ ]:
def calculate_regionprops(label_image):
    import math

    # Define additiinal properties
    def circularity(p):
        return 4 * math.pi * p['area'] / (p['perimeter_crofton'] ** 2) # 'perimeter' calculates circularities up to 1.5... crofton seems to be more accurate
    def aspect_ratio(p):
        return p['axis_major_length'] / p['axis_minor_length']

    # Get properties as numpy arrays
    props = regionprops(label_image)
    area_np = np.asarray([p['area'] for p in props])
    solidity_np = np.asarray([p['solidity'] for p in props])
    circularity_np = np.asarray([circularity(p) for p in props])
    aspect_ratio_np = np.asarray([aspect_ratio(p) for p in props])
    return area_np, solidity_np, circularity_np, aspect_ratio_np

## Plot histograms for the different shape descriptors

In [ ]:
area_array = []
solidity_array = []
circularity_array = []
aspect_ratio_array = []
for image_name in all_images:
    # Load original image
    #img_filename = os.path.join(images_path, image_name)
    #image = io.imread(img_filename)
        
    # Load label image
    lbl_filename = os.path.join(labels_path, image_name)
    labels = io.imread(lbl_filename)
    
    # Transform to label matrix
    labels = label(labels)

    area, sol, circ, as_re = calculate_regionprops(labels)
    area_array = np.concatenate((area_array, area))
    solidity_array = np.concatenate((solidity_array, sol))
    circularity_array = np.concatenate((circularity_array, circ))
    aspect_ratio_array = np.concatenate((aspect_ratio_array, as_re))

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
fig.suptitle('Object distribution for different shape descriptors', fontsize=16)
ax[0].set_title('Area')
ax[0].set_ylabel('Number of objects')
ax[0].hist(area_array, bins=100)
ax[1].set_title('Solidity')
ax[1].set_ylabel('Number of objects')
ax[1].hist(solidity_array, bins=100)
ax[2].set_title('Circularity')
ax[2].set_ylabel('Number of objects')
ax[2].hist(circularity_array, bins=100)
ax[3].set_title('Aspect Ratio')
ax[3].set_ylabel('Number of objects')
ax[3].hist(aspect_ratio_array, bins=100)
plt.show()

## Create color-coded images for the different shape descriptors

In [ ]:
def feature_mask(label_image, feature_list, separators):
    class_list = []

    props = regionprops(label_image)
    for i in range(len(props)):
        label_id = props[i].label # gets the label id of the current region
        
        if(feature_list[i]<separators[0]):
            class_mask = np.where(label_image == label_id, 1, 0)
        elif(feature_list[i]<separators[1]):
            class_mask = np.where(label_image == label_id, 2, 0)
        elif(feature_list[i]<separators[2]):
            class_mask = np.where(label_image == label_id, 3, 0)
        else:
            class_mask = np.where(label_image == label_id, 4, 0)
        
        class_list.append(class_mask)

    # reconstructs the label image, now containing the eroded labels
    class_stack = np.stack(class_list) # creates stack from list of images (numpy arrays)
    feature_labels = np.max(class_stack, axis = 0) # calculates the maximum projection to get back a 2D, labelled image
    return feature_labels

In [ ]:
def color_code_image(image_name, image, label_image, plot_figure):
    from skimage.color import label2rgb

    # define RGB colors from Tableau 10 (default in matplotlib)
    t10_blue = [31/255, 119/255, 180/255]
    t10_orange = [255/255, 127/255, 14/255]
    t10_green = [44/255, 160/255, 44/255]
    t10_red = [214/255, 39/255, 40/255]

    # list of colors
    colors=[t10_blue, t10_orange, t10_green, t10_red]

    # create RGB image with color-labels over the original, grayscale image
    rgb_labels = label2rgb(
        label=label_image,
        image=image,
        colors=colors,
        alpha=0.7,
        bg_label=0,
        bg_color=None
        )

    if plot_figure:
        # plot original and color-coded images
        fig, ax = plt.subplots(1,2, figsize=(14,7))
        fig.suptitle(image_name)
        ax[0].imshow(image, cmap='gray')
        ax[1].imshow(rgb_labels)
        plt.show()

    return rgb_labels

In [ ]:
for image_name in all_images:
    
    # Load original image
    img_filename = os.path.join(images_path, image_name)
    image = io.imread(img_filename)
    
    # Load label image
    lbl_filename = os.path.join(labels_path, image_name)
    labels = io.imread(lbl_filename)

    # Transform to label matrix
    labels = label(labels)

    # Calculate region properties
    area_list, sol_list, circ_list, as_re_list = calculate_regionprops(labels)

    # Get feature color-coded images
    labels_area = feature_mask(labels, area_list, [500, 2000, 5000])
    labels_solidity = feature_mask(labels, sol_list, [0.93, 0.95, 0.98])
    labels_circularity = feature_mask(labels, circ_list, [0.85, 0.9, 0.95])
    labels_aspect_ratio = feature_mask(labels, as_re_list, [1.1, 1.2, 1.4])

    #color_code_area = color_code_image(image, labels_area)
    #imageio.imwrite(os.path.join(r'C:\Users\Pau\Desktop\Neurosphere Annotations\export\color-coded\area', image_name), color_code_area)
    color_code_aspect_ratio = color_code_image(image_name, image, labels_aspect_ratio, True)
    imageio.imwrite(os.path.join(r'C:\Users\Pau\Desktop\Neurosphere Annotations\export\color-coded\aspect_ratio', str(Path(image_name).stem + '.png')), color_code_aspect_ratio)
    #color_code_image(image, labels_circularity)
    #color_code_image(image, labels_aspect_ratio)